In [45]:
class ArithmeticEncoder(object):
    def __init__(self, bitlen):
        self.bit_prec = bitlen
        self.max_range = 1 << self.bit_prec
        self.mask = self.max_range - 1 
        self.renorm= self.max_range >> 1
        self.low = 0
        self.high = self.mask
        self.s = 0

    def update(self, sym, c):
        low = self.low 
        high = self.high
        range = high - low + 1 
        total = (c[-1])
        symlow = c[sym]
        symhigh = c[sym+1]
#         print('total:', total)
#         print('low:', symlow)
#         print('high:', symhigh)

        newlow = low + symlow*range // total
        newhigh = low + symhigh*range // total
        self.low = int(newlow)
#         print('newlow', self.low)
        self.high = int(newhigh)
#         print('newhigh', self.high)
        
        range = self.high - self.low
        #renormalization
        
        while(range < self.renorm):
            self.low = self.low * 2
            range = range*2
            self.s = self.s + 1
#             print(self.s)
            self.high = range + self.low
#         print("renorm low:", self.low)
#         print("renorm_high:", self.high)
#         print("renorm range:", range)

    def write(self, c, sym):
        self.update(c,sym)
        return [self.low, self.s]
        
    def finish(self):
        self.output.write(1)

In [47]:
#test bit stream

import numpy as np
from bitstream import BitStream
from numpy import array
in_stream = array(['a','c', 'g', 'g', 'c','t','n'])
prob = array([0.4, 0.2, 0.2, 0.1, 0.1])
bs = 1024
print("symbol stream:", in_stream)
print("probability:", prob)
c = []
for i in range(len(prob)+1):
    c.append(sum(prob[0:i]))
print("cumulative prob:", c)
sym = []

for x, letter in enumerate(in_stream): 
    if letter == 'a':
        sym.append(0)
    elif letter =='c':
        sym.append(1)
    elif letter == 'g':
        sym.append(2)
    elif letter == 't':
        sym.append(3)
    else: 
        sym.append(4)


enc = ArithmeticEncoder(8)
for j in range(len(sym)):
    new = enc.write(sym[j], c)
    print("[low, s]:", new)

low_final = new[0]/2**8
s_final = new[1]

bitstream = bin(int(low_final))
print(bitstream)




symbol stream: ['a' 'c' 'g' 'g' 'c' 't' 'n']
probability: [0.4 0.2 0.2 0.1 0.1]
cumulative prob: [0, 0.4, 0.6000000000000001, 0.8, 0.9, 1.0]
[low, s]: [0, 1]
[low, s]: [328, 3]
[low, s]: [1708, 5]
[low, s]: [14296, 8]
[low, s]: [57528, 10]
[low, s]: [461352, 13]
[low, s]: [7383712, 17]
0b111000010101010
